## Using SEP to detect objects in an image
Import necessary packages, use astropy.io.fits instead of fitsio. (Audrey)

In [ ]:
%matplotlib inline
import numpy as np
import sep
import matplotlib.pyplot as plt
from astropy.io import fits
from matplotlib import rcParams
from matplotlib.patches import Ellipse

In [ ]:
rcParams['figure.figsize'] = [10., 8.]

#### Open the FITS file. Read image (downloaded from the sep GitHub account) into standard 2-D numpy array. (Audrey)

In [ ]:
fname = "sep_example_image.fits"
hdu_list = fits.open(fname)
hdu_list.info()
image_data = hdu_list[0].data

#### Close the FITS file. (Audrey)

In [ ]:
hdu_list.close()

#### Show the image. (Audrey)

In [ ]:
m = np.mean(image_data)
s = np.std(image_data)

plt.imshow(image_data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()
plt.savefig('raw_image.png');

## Background Subtraction

#### Measure a spatially varying background on the image (Audrey)

In [ ]:
bkg = sep.Background(image_data)

#### Get a "global" mean and noise of the image background (Audrey)

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

#### Evaluate background as 2-D array, same size as original image (Audrey)

In [ ]:
bkg_image = bkg.back()

#### Show the background (Audrey)

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()
plt.savefig('background.png');

#### Evaluate the background noise as 2-D array, same size as original image (Audrey)

In [ ]:
bkg_rms = bkg.rms()

#### Show the background noise (Audrey)

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()
plt.savefig('background_noise.png');

#### Subtract the background (Audrey)

In [ ]:
image_data_sub = image_data - bkg

## Object Detection

#### Set the detection threshold to a constant value (Justine)

In [ ]:
objects = sep.extract(image_data_sub, 1.5, err=bkg.globalrms)

n = len(objects)
print('%d objects detected'%n)

#### Plot background-subtracted image with a red ellipse for each object (Justine)

In [ ]:
fig, ax = plt.subplots()
m = np.mean(image_data_sub)
s= np.std(image_data_sub)
im = ax.imshow(image_data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')

for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
    
plt.savefig('detected_objects.png')

## Aperture photometry
#### Perform simple circular aperture photometry with a 3 pixel radius at the locations of the objects (Justine)

In [ ]:
flux, fluxerr, flag = sep.sum_circle(image_data_sub, objects['x'], objects['y'], 3.0, err=bkg.globalrms, gain=1.0)

#### Show the objects results (Justine)

In [ ]:
for i in range(n):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))